In [ ]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
import matplotlib as mpl
from matplotlib.colors import LogNorm

import boost_histogram as bh
import pickle as pl

import scipy.stats as stats
import scipy.optimize as opt
from pygama import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
import cage_utils
plt.rcParams['figure.figsize'] = [14, 10]

In [ ]:
run = 64 #44 #70 #64
dsp_id = '02'
user = True
hit =True
cal = True
lowE=False
etype = 'trapEftp_cal'

corr_DCR=True
corr_AoE=True
corr_ToE=True

norm = True
cut = True


dsp_list = ['energy', 'trapEftp', 'trapEmax', 'trapEftp_cal', 'bl','bl_sig', 'bl_slope', 'AoE', 'dcr', "tp_0", "tp_02", "tp_05", "tp_10", "tp_20", "tp_30", "tp_40", "tp_50", "tp_60", "tp_70", "tp_80", "tp_90", "tp_96", 'tp_max', 'ToE', 'log_tail_fit_slope', 'wf_max', 'wf_argmax', 'trapE_argmax', 'lf_max']

df_raw, dg, runtype, rt_min, radius, angle_det, rotary = cage_utils.getDataFrame(run, user=user, hit=hit, cal=cal, dsp_list=dsp_list, lowE=lowE)



In [ ]:
muon_cut = f'energy < 11000000 and wf_max<17000 and trapEmax < 17000 and trapEftp_cal < 6000'
df_raw = df_raw.query(muon_cut).copy()

In [ ]:
fig, ax = plt.subplots()
elo, ehi, epb = 50, 6000, 1
e_bins = int((ehi-elo)/epb)
wflo, wfhi = 50, 17000
wf_bins = int((wfhi - wflo )/10)
fig.suptitle(f'\nwf_max vs Energy', horizontalalignment='center', fontsize=16)
wf_maxVEnergy, xedges, yedges = np.histogram2d(df_raw['wf_max'], df_raw['trapEftp_cal'], bins=[wf_bins, e_bins], range=([wflo, wfhi], [elo, ehi]))
X, Y = np.mgrid[wflo:wfhi:wf_bins*1j, elo:ehi:e_bins*1j]

pcm = plt.pcolormesh(X, Y, wf_maxVEnergy,norm=LogNorm())
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel('wf_max', fontsize=16)
ax.set_ylabel('trapEftp_cal (keV)', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)
# plt.xlim(0, 1000)
# plt.ylim(0, 200)

# plt.ylim(0, 100)
# plt.xlim(0, 500)
plt.show()

In [ ]:
median, xedges, binnumber = stats.binned_statistic(df_raw['wf_max'], df_raw['trapEftp_cal'], statistic = 'mean', bins = wf_bins) #, range = (wflo, wfhi)


en_bin_centers = pgh.get_bin_centers(xedges)

idx = np.isfinite(en_bin_centers) & np.isfinite(median)


fit_raw, cov = np.polyfit(en_bin_centers[idx], median[idx], deg=1, cov=True)

const = fit_raw[0]
offset = fit_raw[1]
err = np.sqrt(np.diag(cov))

In [ ]:
fig, ax = plt.subplots()


elo, ehi, epb = 0, 5500, 1
e_bins = int((ehi-elo)/epb)
wflo, wfhi = 50, 17000
wf_bins = int((wfhi - wflo )/10)
fig.suptitle(f'\nwf_max vs Energy', horizontalalignment='center', fontsize=16)
tp0Vtpmax_hist_norm, xedges, yedges = np.histogram2d(df_raw['wf_max'], df_raw['trapEftp_cal'], bins=[wf_bins, e_bins], range=([wflo, wfhi], [elo, ehi]))
X, Y = np.mgrid[wflo:wfhi:wf_bins*1j, elo:ehi:e_bins*1j]

pcm = plt.pcolormesh(X, Y, tp0Vtpmax_hist_norm, norm=LogNorm())
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel('wf_max', fontsize=16)
ax.set_ylabel('trapEftp_cal (keV)', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)
# plt.xlim(200, 1000)
# plt.ylim(0, 400)
y = const*en_bin_centers + offset
plt.plot(en_bin_centers, y, 'r')

# plt.ylim(2000, 5000)
# plt.ylim(40, 80)
# plt.xlim(50, 500)

plt.ylim(0, 100)
plt.xlim(0, 500)
plt.show()

In [ ]:
wf_cut = f'trapEftp_cal > ({const}*wf_max + {offset})*(1+0.04) and trapEftp_cal > 50 and trapEftp_cal < 70' # 
wf_cut_pass = len(df_raw.query(wf_cut).copy())
print(wf_cut_pass)

In [ ]:
times, wfs_wf_max = cage_utils.get_wfs(df_raw, dg, wf_cut , nwfs = 100)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 10))
ax = plt.axes()

# set up colorbar to plot waveforms of different energies different colors
colors = plt.cm.viridis(np.linspace(0, 1, len(wfs_wf_max)))
c = np.arange(0, len(wfs_wf_max))
norm = mpl.colors.Normalize(vmin=c.min(), vmax=c.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.plasma)
cmap.set_array([])

for n in range(len(wfs_wf_max)):
    plt.plot(times, wfs_wf_max[n][:len(wfs_wf_max[n])-1], c=cmap.to_rgba(n))

cb = fig.colorbar(cmap)
cb.set_label("Energy", ha = 'right', va='center', rotation=270, fontsize=20)
cb.ax.tick_params(labelsize=18)

ax.text(0.95, 0.83, f'{wf_cut}', verticalalignment='top',
                        horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14,
                        bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

plt.title(f'run {run}: r = {radius} mm \ntheta = {angle_det} deg')

#     plt.xlim(3800, 8000)
#     plt.ylim(0.4, 1.01)
plt.setp(ax.get_xticklabels(), fontsize=16)
plt.setp(ax.get_yticklabels(), fontsize=16)
# plt.title(f'60 keV waveforms for run {run}\nr = {radius} mm; theta = {angle_det} deg', fontsize=20)
plt.xlabel('clock cycles', fontsize=20)
# plt.xlim(3900, 4100)
# plt.xlim(3600, 5000)
# plt.ylim(0,200)

In [ ]:
wf_cut_hi = f'trapEftp_cal < ({const}*wf_max + {offset})*(1-0.04) and trapEftp_cal > 50 and trapEftp_cal < 70 ' #
wf_cut_pass_hi = len(df_raw.query(wf_cut_hi).copy())
print(wf_cut_pass_hi)

In [ ]:
times, wfs_wf_max_hi = cage_utils.get_wfs(df_raw, dg, wf_cut_hi , nwfs = 100)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 10))
ax = plt.axes()

# set up colorbar to plot waveforms of different energies different colors
colors = plt.cm.viridis(np.linspace(0, 1, len(wfs_wf_max_hi)))
c = np.arange(0, len(wfs_wf_max_hi))
norm = mpl.colors.Normalize(vmin=c.min(), vmax=c.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.plasma)
cmap.set_array([])

for n in range(len(wfs_wf_max_hi)):
    plt.plot(times, wfs_wf_max_hi[n][:len(wfs_wf_max_hi[n])-1], c=cmap.to_rgba(n))

cb = fig.colorbar(cmap)
cb.set_label("Energy", ha = 'right', va='center', rotation=270, fontsize=20)
cb.ax.tick_params(labelsize=18)

ax.text(0.95, 0.83, f'{wf_cut_hi}', verticalalignment='top',
                        horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14,
                        bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

plt.title(f'run {run}: r = {radius} mm \ntheta = {angle_det} deg')

#     plt.xlim(3800, 8000)
#     plt.ylim(0.4, 1.01)
plt.setp(ax.get_xticklabels(), fontsize=16)
plt.setp(ax.get_yticklabels(), fontsize=16)
# plt.title(f'60 keV waveforms for run {run}\nr = {radius} mm; theta = {angle_det} deg', fontsize=20)
plt.xlabel('clock cycles', fontsize=20)
# plt.xlim(3900, 4100)
# plt.xlim(3600, 5000)
# plt.ylim(0,200)

In [ ]:
wf_max_cut = f'({const}*wf_max + {offset})*(1-0.04) < trapEftp_cal < ({const}*wf_max + {offset})*(1+0.04)'

df_raw_tot = len(df_raw)
cut_pass = len(df_raw.query(wf_max_cut).copy())
print(f'{cut_pass} events passing cut \n{(cut_pass/df_raw_tot)*100}%')

df= df_raw.query(wf_max_cut).copy()
df_60 = df.query('trapEftp_cal > 50 and trapEftp_cal < 70')

In [ ]:
df_raw['tp05max'] = df_raw['tp_max'] - df_raw['tp_05']
df_raw['tp0max'] = df_raw['tp_max'] - df_raw['tp_0']
df_raw['tp0max_E'] = df_raw['tp0max']/df_raw['trapEmax']
df_raw['tp0max_wf'] = df_raw['tp0max']/df_raw['wf_max'] 
df_raw['ftp_max'] = df_raw['trapEftp']/df_raw['trapEmax']

In [ ]:
fig, ax = plt.subplots()
tlo, thi, tpb = 0, 35, 1
t_bins = int((thi-tlo)/tpb)
wflo, wfhi = 50, 8000
wf_bins = int((wfhi - wflo )/10)
fig.suptitle(f'\ntp0max/wf_max vs wf_max', horizontalalignment='center', fontsize=16)
tp0Vtpmax_hist_norm, xedges, yedges = np.histogram2d(df_raw['wf_max'], df_raw['tp0max_wf'], bins=[wf_bins, t_bins], range=([wflo, wfhi], [tlo, thi]))
X, Y = np.mgrid[wflo:wfhi:wf_bins*1j, tlo:thi:t_bins*1j]

pcm = plt.pcolormesh(X, Y, tp0Vtpmax_hist_norm, norm=LogNorm())
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel('wf_max', fontsize=16)
ax.set_ylabel('tp0max/wf_max (arb)', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)
plt.xlim(0, 700)
plt.show()

In [ ]:
fig, ax = plt.subplots()
tlo, thi, tpb = 50, 5500, 50
t_bins = int((thi-tlo)/tpb)
wflo, wfhi = 10, 6000
wf_bins = int((wfhi - wflo )/0.5)
fig.suptitle(f'\ntp0max/wf_max vs energy', horizontalalignment='center', fontsize=16)
tp0Vtpmax_hist_norm, xedges, yedges = np.histogram2d(df_raw['trapEftp_cal'], df_raw['tp05max'], bins=[wf_bins, t_bins], range=([wflo, wfhi], [tlo, thi]))
X, Y = np.mgrid[wflo:wfhi:wf_bins*1j, tlo:thi:t_bins*1j]

pcm = plt.pcolormesh(X, Y, tp0Vtpmax_hist_norm, norm=LogNorm())
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel('energy (keV)', fontsize=16)
ax.set_ylabel('tp05max (arb)', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)
# plt.xlim(40, 80)
plt.show()

In [ ]:
fig, ax = plt.subplots()
tlo, thi, tpb = 50, 5500, 50
t_bins = int((thi-tlo)/tpb)
wflo, wfhi = 10, 6000
wf_bins = int((wfhi - wflo )/0.5)
fig.suptitle(f'\ntp0max/wf_max vs energy', horizontalalignment='center', fontsize=16)
tp0Vtpmax_hist_norm, xedges, yedges = np.histogram2d(df['trapEftp_cal'], df['tp05max'], bins=[wf_bins, t_bins], range=([wflo, wfhi], [tlo, thi]))
X, Y = np.mgrid[wflo:wfhi:wf_bins*1j, tlo:thi:t_bins*1j]

pcm = plt.pcolormesh(X, Y, tp0Vtpmax_hist_norm, norm=LogNorm())
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel('energy (keV)', fontsize=16)
ax.set_ylabel('tp05max (arb)', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)
# plt.xlim(40, 80)
plt.show()

In [ ]:
fig, ax = plt.subplots()
tlo, thi, tpb = 0, 5500, 10
t_bins = int((thi-tlo)/tpb)
wflo, wfhi = 10, 6000
wf_bins = int((wfhi - wflo )/1)
fig.suptitle(f'\ntp0max vs energy', horizontalalignment='center', fontsize=16)
tp0Vtpmax_hist_norm, xedges, yedges = np.histogram2d(df_raw['trapEftp_cal'], df_raw['tp0max'], bins=[wf_bins, t_bins], range=([wflo, wfhi], [tlo, thi]))
X, Y = np.mgrid[wflo:wfhi:wf_bins*1j, tlo:thi:t_bins*1j]

pcm = plt.pcolormesh(X, Y, tp0Vtpmax_hist_norm, norm=LogNorm())
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel('energy (keV)', fontsize=16)
ax.set_ylabel('tp0max', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)
# plt.xlim(10, 500)
plt.show()

In [ ]:
tp_cut = f'tp05max > 3000 and trapEftp_cal >55 and trapEftp_cal <65 '
print(f'{len(df_raw.query(tp_cut).copy())} events passing cut')



In [ ]:
times, wfs_tp_cut = cage_utils.get_wfs(df_raw, dg, tp_cut, nwfs = 100)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 10))
ax = plt.axes()

# set up colorbar to plot waveforms of different energies different colors
colors = plt.cm.viridis(np.linspace(0, 1, len(wfs_tp_cut)))
c = np.arange(0, len(wfs_tp_cut))
norm = mpl.colors.Normalize(vmin=c.min(), vmax=c.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.plasma)
cmap.set_array([])

for n in range(len(wfs_tp_cut)):
    plt.plot(times, wfs_tp_cut[n][:len(wfs_tp_cut[n])-1], c=cmap.to_rgba(n))

cb = fig.colorbar(cmap)
cb.set_label("Energy", ha = 'right', va='center', rotation=270, fontsize=20)
cb.ax.tick_params(labelsize=18)

ax.text(0.95, 0.83, f'{tp_cut}', verticalalignment='top',
                        horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14,
                        bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

plt.title(f'run {run}: r = {radius} mm \ntheta = {angle_det} deg')

#     plt.xlim(3800, 8000)
#     plt.ylim(0.4, 1.01)
plt.setp(ax.get_xticklabels(), fontsize=16)
plt.setp(ax.get_yticklabels(), fontsize=16)
# plt.title(f'60 keV waveforms for run {run}\nr = {radius} mm; theta = {angle_det} deg', fontsize=20)
plt.xlabel('clock cycles', fontsize=20)
# plt.xlim(3900, 4100)
# plt.xlim(3600, 5000)
# plt.ylim(0,200)

In [ ]:
fig, ax = plt.subplots()
wflo, wfhi, wfpb = 41000, 45500, 50
wf_bins = int((wfhi-wflo)/wfpb)

elo, ehi, epb= 500, 5500, 10
e_bins  = int((ehi-elo)/epb)
fig.suptitle(f'\ntrapE_argmax vs energy', horizontalalignment='center', fontsize=16)
energy_vs_trapE_argmax, xedges, yedges = np.histogram2d(df_raw['trapEftp_cal'], df_raw['trapE_argmax'], bins=[e_bins, wf_bins], range=([elo, ehi], [wflo, wfhi]))
X, Y = np.mgrid[elo:ehi:e_bins*1j, wflo:wfhi:wf_bins*1j]

pcm = plt.pcolormesh(X, Y, energy_vs_trapE_argmax, norm=LogNorm())
cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel('energy', fontsize=16)
ax.set_ylabel('trapE_argmax', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=12)
plt.setp(ax.get_yticklabels(), fontsize=14)
# plt.xlim(elo, ehi)
plt.show()
# print(ToElo, ToEhi)

In [ ]:
# cut_trapE_argmax = f' trapE_argmax > 44900 and trapE_argmax < 45000' #and trapE_argmax < 44500
cut_trapE_argmax = f' trapE_argmax > 44900 and trapEftp_cal < 500' #and trapEftp_cal > 1000

print(f'{len(df_raw.query(cut_trapE_argmax ).copy())} events passing cut')

times, wfs_trapE_argmax = cage_utils.get_wfs(df_raw, dg, cut_trapE_argmax , nwfs = 100)

In [ ]:
# fig, ax = plt.subplots(figsize=(9,8))
ax = plt.axes()

# set up colorbar to plot waveforms of different energies different colors
colors = plt.cm.viridis(np.linspace(0, 1, len(wfs_trapE_argmax)))
c = np.arange(0, len(wfs_trapE_argmax))
norm = mpl.colors.Normalize(vmin=c.min(), vmax=c.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.plasma)
cmap.set_array([])

for n in range(len(wfs_trapE_argmax)):
    plt.plot(times, wfs_trapE_argmax[n][:len(wfs_trapE_argmax[n])-1], c=cmap.to_rgba(n))

# cb = fig.colorbar(cmap, ticks=(57.5, 59.5))
cb.set_label("Energy", ha = 'right', va='center', rotation=270, fontsize=20)
cb.ax.tick_params(labelsize=18)

ax.text(0.95, 0.83, f'{cut_trapE_argmax}', verticalalignment='top',
                        horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14,
                        bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

plt.title(f'run {run}: r = {radius} mm \ntheta = {angle_det} deg')

#     plt.xlim(3800, 8000)
#     plt.ylim(0.4, 1.01)
plt.setp(ax.get_xticklabels(), fontsize=16)
plt.setp(ax.get_yticklabels(), fontsize=16)
# plt.title(f'60 keV waveforms for run {run}\nr = {radius} mm; theta = {angle_det} deg', fontsize=20)
plt.xlabel('clock cycles', fontsize=20)
# plt.xlim(3800, 4200)
# plt.ylim(8000, 12000)
# plt.ylim(0,500)